<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Evaluation/Exact_Match_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize Libraries


In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 10.4 MB/s eta 0:00:00


In [4]:
import torch
from torchmetrics.classification import ExactMatch
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

# Reading the Dataset

In [6]:
path = "/content/drive/MyDrive/Bachelor\'s project/bug_evaluation_dataset.csv"
df = pd.read_csv(path)
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: "/content/drive/MyDrive/Bachelor's project/bug_evaluation_dataset.csv"

In [ ]:
import torch
from torchmetrics.classification import ExactMatch
from sklearn.preprocessing import LabelEncoder

# Sample code snippets
code1 = """
void setup() {
    pinMode(LED_BUILTIN, OUTPUT);
    digitalWrite(LED_BUILTIN, HIGH);
}
"""

code2 = """
void setup() {
    pinMode(LED_BUILTIN, OUTPUT);
    digitalWrite(LED_BUILTIN, HIH);
}
"""

# Tokenize by splitting on spaces, special characters can be handled more sophisticatedly
def tokenize_code(code):
    return code.split()

# Convert token sequences to labels (integers)
def tokenize_and_encode(code1, code2):
    tokens1 = tokenize_code(code1)
    tokens2 = tokenize_code(code2)
    print(tokens1)

    # Combine tokens to get a consistent vocabulary
    all_tokens = tokens1 + tokens2
    print(all_tokens)

    # Use LabelEncoder to convert tokens to integer labels
    label_encoder = LabelEncoder()
    label_encoder.fit(all_tokens)

    # Convert both code snippets to sequences of labels
    token_labels1 = label_encoder.transform(tokens1)
    token_labels2 = label_encoder.transform(tokens2)
    print(token_labels1)
    print(token_labels2)

    return torch.tensor([token_labels1]), torch.tensor([token_labels2])

# Get tokenized and encoded versions of the two code snippets
preds, target = tokenize_and_encode(code1, code2)
print(preds)
print(target)

# Flatten the lists (in case they are lists of lists)
flat_preds = [item for sublist in preds.tolist() for item in sublist]
flat_target = [item for sublist in target.tolist() for item in sublist]


# Calculate number of unique classes (tokens)
num_classes = len(set(flat_preds + flat_target))


# Initialize ExactMatch metric
metric = ExactMatch(task="multiclass", num_classes=num_classes, multidim_average='global')

# Compute the Exact Match score
exact_match_score = metric(flat_preds, flat_target)


print(f"Exact Match Accuracy: {exact_match_score.item() * 100:.2f}%")


['void', 'setup()', '{', 'pinMode(LED_BUILTIN,', 'OUTPUT);', 'digitalWrite(LED_BUILTIN,', 'HIGH);', '}']
['void', 'setup()', '{', 'pinMode(LED_BUILTIN,', 'OUTPUT);', 'digitalWrite(LED_BUILTIN,', 'HIGH);', '}', 'void', 'setup()', '{', 'pinMode(LED_BUILTIN,', 'OUTPUT);', 'digitalWrite(LED_BUILTIN,', 'HIH);', '}']
[6 5 7 4 2 3 0 8]
[6 5 7 4 2 3 1 8]
tensor([[6, 5, 7, 4, 2, 3, 0, 8]])
tensor([[6, 5, 7, 4, 2, 3, 1, 8]])


AttributeError: 'list' object has no attribute 'ndim'